# Project Steps-by-step 

### Install Azure CLI

https://learn.microsoft.com/en-us/cli/azure/install-azure-cli-linux?view=azure-cli-latest&pivots=dnf

### Azure auth

In [ ]:
az login
az account set --subscription "<SUBSCRIPTION_ID>"

### Create a Resouce group

In [ ]:
az group create \
  --name eraki-containerApps-us1-rg-1001 \
  --location eastus
  --tags Project=containerAppsPr Environment=Dev

### Create a user-assigned managed identity

In [ ]:
az identity create \
  --name terraform-uami \
  --resource-group eraki-containerApps-us1-rg-1001 \
  --location eastus \
  --tags Project=containerAppsPr Environment=Dev

Copy the JSON output somewhere, will be using in the next comand.

### Assign RBAC to user-assigned managed identity

In [ ]:
az role assignment create \
  --assignee-object-id <principalId> \
  --role Contributor \
  --scope /subscriptions/<your-subscription-id>

### Configure Azure DevOps service connection

**Create an Azure DevOps Service Connection**

- Go to your Azure DevOps Project:

- Project Settings → Service Connections → New Service Connection

- Choose Azure Resource Manager → Manage Identity.

- Specify you Subscription, Resource group, and Managed identity.

- Define Azure scope.

- Define Service connection name, then Save.

### Configure Azure Storage account for state file

In [ ]:
# Variables
STORAGE_ACCOUNT_NAME="erakitfstateaccount$RANDOM"
CONTAINER_NAME="containerappstfstate"

# Create resource group
az group create --name eraki-containerApps-us1-rg-1001 --location eastus

# Create storage account
az storage account create \
  --name $STORAGE_ACCOUNT_NAME \
  --resource-group eraki-containerApps-us1-rg-1001 \
  --sku Standard_LRS \
  --encryption-services blob

# Create blob container
az storage container create \
  --name $CONTAINER_NAME \
  --account-name $STORAGE_ACCOUNT_NAME


### Configure Terraform provider

Run the following Commands

In [ ]:
export ARM_SUBSCRIPTION_ID=<SUB_ID>
export ARM_CLIENT_ID=<CLIENT_ID>

This will be ignored while using Azure DevOps, use Environment variables instead.

In [ ]:
terraform {
  required_providers {
    azurerm = {
      source = "hashicorp/azurerm"
      version = "4.43.0"
    }
  }
    backend "azurerm" {
    resource_group_name  = "eraki-containerApps-us1-rg-1001"
    storage_account_name = "erakitfstateaccount17108"
    container_name       = "containerappstfstate"
    key                  = "containerApps-terraform.tfstate"
    use_msi              = true # use Managed Identity
  }
}

provider "azurerm" {
  
  features {}

  use_msi = true  # using user-assigned managed identity

  # Set the client_id of your User-assigned MI
  client_id = <"client_id">

  subscription_id = <"subscription_id">
}



### Configure Azure Pipeline


In [ ]:
trigger:
- main

pool:
  name: 'Self-Hosted-Agent-Pool'  # Must point to your VM with the Managed Identity

variables:
  ARM_CLIENT_ID: 'CLIENT_ID'
  ARM_SUBSCRIPTION_ID: 'subscription_id'

steps:
- task: TerraformInstaller@1
  inputs:
    terraformVersion: '1.8.0'

- script: terraform init
  displayName: 'Terraform Init'

- script: terraform plan -out=tfplan
  displayName: 'Terraform Plan'

- script: terraform apply tfplan
  displayName: 'Terraform Apply'
